#### Names of people in the group

Please write the names of the people in your group in the next cell.

Name of person A Karl Edvin Undheim

Name of person B

In [0]:
# Deleting tables left from previous runs in case they still exist after deleting an inactive cluster
dbutils.fs.rm("/user", recurse=True)

Out[1]: True

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading PySpark modules that we need
import unittest
from collections import Counter
from pyspark.sql import DataFrame
from pyspark.sql.types import *

#### Subtask 1: defining the schema for the data
Typically, the first thing to do before loading the data into a Spark cluster is to define the schema for the data. Look at the schema for 'badges' and try to define the schema for other tables similarly.

In [0]:
# In this task I used the "discription of the data" document we were provided to find each of the files' content. For some attributes I was unsure of its type. To clarify this I went to "Data" and created a table for the file which had the data I was unsure of. In this table it is clear which type it is. An example was the reputation in users, which I interpreted as either string or integer.
# Next I defined the schemas as shown in badges_schema

# Defining a schema for 'badges' table
badges_schema = StructType([StructField('UserId', IntegerType(), False),
                            StructField('Name', StringType(), False),
                            StructField('Date', TimestampType(), False),
                            StructField('Class', IntegerType(), False)])

# Defining a schema for 'posts' table
posts_schema = StructType([StructField('Id', IntegerType(), False),
                           StructField('ParentId', IntegerType(), False),
                           StructField('PostTypeId', IntegerType(), False),
                           StructField('CreationDate', TimestampType(), False),
                           StructField('Score', IntegerType(), False),
                           StructField('ViewCount', IntegerType(), False),
                           StructField('Body', StringType(), False),
                           StructField('OwnerUserId', IntegerType(), False),
                           StructField('LastActivityDate', TimestampType(), False),
                           StructField('Title', StringType(), False),
                           StructField('Tags', StringType(), False),
                           StructField('AnswerCount', IntegerType(), False),
                           StructField('CommentCount', IntegerType(), False),
                           StructField('FavoriteCount', IntegerType(), False),
                           StructField('CloseDate', TimestampType(), False)])

# Defining a schema for 'users' table
users_schema = StructType([StructField('Id', IntegerType(), False),
                           StructField('Reputation', IntegerType(), False),
                           StructField('CreationDate', TimestampType(), False),
                           StructField('DisplayName', StringType(), False),
                           StructField('LastAccessDate', TimestampType(), False),
                           StructField('AboutMe', StringType(), False),
                           StructField('Views', IntegerType(), False),
                           StructField('UpVotes', IntegerType(), False),
                           StructField('DownVotes', IntegerType(), False)])

# Defining a schema for 'comments' table
comments_schema = StructType([StructField('PostId', IntegerType(), False),
                          StructField('Score', IntegerType(), False),
                          StructField('Text', StringType(), False),
                          StructField('CreationDate', TimestampType(), False),
                          StructField('UserId', IntegerType(), False)])

#### Subtask 2: implementing two helper functions
Next, we need to implement two helper functions:
1. 'load_csv' that as input argument receives path for a CSV file and a schema and loads the CSV pointed by the path into a Spark DataFrame and returns the DataFrame;
2. 'save_df' receives a Spark DataFrame and saves it as a Parquet file on DBFS.

Note that the column separator in CSV files is TAB character ('\t') and the first row includes the name of the columns. 

BTW, DBFS is the name of the distributed filesystem used by Databricks Community Edition to store and access data.

In [0]:
def load_csv(source_file: "path for the CSV file to load", schema: "schema for the CSV file being loaded as a DataFrame") -> DataFrame:
  # Here I used the spark operation to read from a csv file.
  # delimiter defines the column separator, which is \t as stated.
  df = spark.read.load(source_file, format="csv", delimiter='\t', schema=schema, header="true")
  return df;


def save_df(df: "DataFrame to be saved", table_name: "name under which the DataFrame will be saved") -> None:
  df.write.parquet("user/hive/warehouse/"+table_name)
  return None;
    ## YOUR IMPLEMENTATION ##

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully. Moreover, at the end there should be four Parquet files named 'badges', 'comments', 'posts', and 'users' in '/user/hive/warehouse'.

Note that we assumed that the data for the project has already been stored on DBFS on the '/FileStore/tables/' path. (I mean as 'badges_csv.gz', 'comments_csv.gz', 'posts_csv.gz', and 'users_csv.gz'.)

In [0]:
%%unittest_main
class TestTask1(unittest.TestCase):
   
    # test 1
    def test_load_badges(self):
        result = load_csv(source_file="/FileStore/tables/badges_csv.gz", schema=badges_schema)
        self.assertIsNotNone(result, "Badges dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 105640, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['UserId', 'Name', 'Date', 'Class']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 2
    def test_load_posts(self):
        result = load_csv(source_file="/FileStore/tables/posts_csv.gz", schema=posts_schema)
        self.assertIsNotNone(result, "Posts dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 61432, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'ParentId', 'PostTypeId', 'CreationDate', 'Score', 'ViewCount', 'Body', 'OwnerUserId',
                                    'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount', 'FavoriteCount',
                                    'CloseDate']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 3
    def test_load_comments(self):
        result = load_csv(source_file="/FileStore/tables/comments_csv.gz", schema=comments_schema)
        self.assertIsNotNone(result, "Comments dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 58735, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower, ['PostId', 'Score', 'Text', 'CreationDate', 'UserId']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    
    # test 4
    def test_load_users(self):
        result = load_csv(source_file="/FileStore/tables/users_csv.gz", schema=users_schema)
        self.assertIsNotNone(result, "Users dataframe did not load successfully")
        self.assertIsInstance(result, DataFrame, "Result type is not of spark.sql.DataFrame")
        self.assertEqual(result.count(), 91616, "Number of records is not correct")

        coulmn_names = Counter(map(str.lower,
                                   ['Id', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate', 'AboutMe',
                                    'Views', 'UpVotes', 'DownVotes']))
        self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)),
                              "Missing column(s) or column name mismatch")
    # test 5
    def test_save_dfs(self):
        dfs = [("/FileStore/tables/users_csv.gz", users_schema, "users"),
               ("/FileStore/tables/badges_csv.gz", badges_schema, "badges"),
               ("/FileStore/tables/comments_csv.gz", comments_schema, "comments"),
               ("/FileStore/tables/posts_csv.gz", posts_schema, "posts")
               ]

        for i in dfs:
            df = load_csv(source_file=i[0], schema=i[1])
            save_df(df, i[2])

Success.....
----------------------------------------------------------------------
Ran 5 tests in 43.511s

OK
Out[7]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'Spark Application', 'SparkSession', 'Transformations', 'Action', and 'Lazy Evaluation' mean in the context of Spark?

Write your descriptions in the next cell.

Note: I will use the term spark datatype when reffering to RDD/DataFrame/Dataset

- A spark application consists of everything involved with running the user's code to compute a result. There are 3 main parts: The driver process, executors and the cluster manager. The driver process consists of a spark session and the user code. This process communicates with the executors and the cluster manager. The executors only execute the code given to them and return the result. The cluster manager oversees spark applications and allocates resources to them.

- The spark session is a connection to a spark cluster. It processes the users code and delegates work to the executors (computing etc). Thus the sparksession provides support for different features such as creating dataframes and performing SQL queries.

- Transformations in spark are operations which return one of the spark datatypes( RDD, DataFrame or dataset). In spark these data structures are immutable which means a transformation will create a new structure, instead of updating the previous one. In short a transformation takes a spark dataype as input and returns a spark datatype as output. Example: RDD -> RDD.

- Actions differ from transformations in that they do not return a spark datatype. Rather they return a value/object of some sort. This value is then returned to the driver process where it can be used further. Example: RDD -> object

- Lazy evaluation means that a spark datatype wont be computed until it is used in an action. This means that if you have a chain of transformations, the process wont execute until an action is called. The advantage of waiting is that spark can then create an optimal way to compute the result by removing or merging redundant transformations. This can increase the efficiency greatly.